## 基础题：改变坐标范围，制作北京、广州的城市发展轨迹图。

北京、广州的坐标范围可以参考以下代码：

cities_coords_beijing = {'Beijing':[(1760,1412),(1840,1576)]}  
cities_coords_guangzhou = {'Guangzhou':[(3723,1033),(3943,1269)]}

#### 思路步骤

根据《灯光定标教案》进行修改，链接：https://www.heywhale.com/mw-org/meteoda/project/62f4aa3b337558d5db3e28a1

主要修改制作城市发展轨迹图中
```python
# 定义城市及其周边区域的坐标
cities_coords = {'Beijing':[(1760,1412),(1840,1576)]}
```
将cities_coords定义为对应需要的城市区域坐标

北京、广州的坐标范围可以参考以下代码：

cities_coords_beijing = {'Beijing':[(1760,1412),(1840,1576)]}
cities_coords_guangzhou = {'Guangzhou':[(3723,1033),(3943,1269)]}



#### 代码

### 预处理：准备数据

In [1]:
# 安装完成后，欢迎点右上角Kernel-保存当前镜像，以便以后直接使用该自定义镜像
!pip install retry pySmartDL -i https://pypi.douban.com/simple/ #从指定镜像下载安装工具包，镜像URL可自行修改

Looking in indexes: https://pypi.douban.com/simple/


In [2]:
# 导入需要的拓展包
from IPython.core.display import display, HTML
import tarfile  
import os
import gzip  
import shutil
import glob
from tqdm import tqdm
import gc
import gdal
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline 
import pandas as pd
from retry import retry
import pandas as pd
from PIL import Image
import imageio
import IPython
from pySmartDL import SmartDL

### 解压数据

In [3]:
# 定义函数：解压文件
@retry()
def gunzip_shutil(source_filepath, block_size=65536):
    # 解压gz文件，提取tif图像
    dest_filepath = source_filepath[:-3]
    with gzip.open(source_filepath, 'rb') as s_file, \
            open(dest_filepath, 'wb') as d_file:
        shutil.copyfileobj(s_file, d_file, block_size)

In [4]:
# 本cell由于数据量较大所以运行耗时较长
# 指定压缩文件gz的目录
inpDir = "/home/mw/input/NTL1992to20131028/stable_lights/"
# 获取所有压缩文件gz的文件名
gzFiles = glob.glob(inpDir + '*.gz')
gzFiles.sort()
# 依次遍历原始压缩文件gz
for gz in tqdm(gzFiles):
    # 从压缩文件gz中提取出stable_lights.avg_vis文件，该文件是tif格式的图像
    gunzip_shutil(gz)

print('全部解压成功！')

100%|██████████| 34/34 [03:47<00:00,  6.70s/it]

全部解压成功！


### 裁剪数据

In [5]:
# 定义函数：读取tif图像成numpy array
@retry()
def readTiff(filename:str):
    ds = gdal.Open(filename)
    image = ds.GetRasterBand(1).ReadAsArray()
    return image,ds.GetProjection(),ds.GetGeoTransform()

@retry()
# 定义函数：根据坐标裁剪子区域
def clipImg(image,proj,geotrans,upperLeftCoords,downRightCoords):
    # 根据坐标提取子矩阵
    r_upperLeft,c_upperLeft = upperLeftCoords
    r_downRight,c_downRight = downRightCoords
    subImage = image[r_upperLeft:r_downRight,c_upperLeft:c_downRight]
    # 设置投影信息
    subProj = proj
    # 设置坐标
    subGeotrans = list(geotrans)
    #图像左上角的坐标为r_upperLeft，c_upperLeft。则该点的经纬度为：
    xGeo = geotrans[0] + c_upperLeft * geotrans[1] + r_upperLeft * geotrans[2]
    yGeo = geotrans[3] + c_upperLeft * geotrans[4] + r_upperLeft * geotrans[5]
    # 左上角经度
    subGeotrans[0] = xGeo
    # 左上角纬度
    subGeotrans[3] = yGeo
    # 垂直空间分辨率(经度间隔)
    subGeotrans[1] = subGeotrans[1]
    # 水平空间分辨率(纬度间隔)
    subGeotrans[5] = subGeotrans[5]
    subGeotrans = tuple(subGeotrans)
    # 
    return subImage,subProj,subGeotrans

@retry()
# 定义函数：保存到磁盘
def outTiff(subArr,subProj,subGeotrans,outputFile):
    # 创建driver
    driver = gdal.GetDriverByName("GTiff")
    # 创建数据集
    outdata = driver.Create(outputFile, subArr.shape[1], subArr.shape[0], 1, gdal.GDT_Float32)
    # 把数组放进数据集
    outdata.GetRasterBand(1).WriteArray(subArr)
    # 设置投影信息和坐标
    outdata.SetProjection(subProj) 
    outdata.SetGeoTransform(subGeotrans)
    # 
    outdata.FlushCache()
    outdata = None
    driver=None
    subArr=None
    print('outTiff: {0}'.format(outputFile))

In [6]:
# 找到所有文件名
globalImgs = glob.glob('/home/mw/input/NTL1992to20131028/stable_lights/*tif')
globalImgs.sort()
# 定义输出路径
# 需提前挂载work目录
outDir = '/home/mw/work/2. clipped dataset/'
if not os.path.isdir(outDir):
    os.mkdir(outDir)
# 遍历所有文件
for fn in tqdm(globalImgs):
    # 读取数据
    image,proj,trans = readTiff(fn)
    # 裁剪图像
    subImage,subProj,subGeotrans = clipImg(image,proj,trans,(2433, 34070),(6971,38082)) #定义选取区间坐标
    del image
    _ = gc.collect()
    # 输出到磁盘
    bn = os.path.basename(fn)
    outFn = outDir + bn
    outTiff(subImage,subProj,subGeotrans,outFn)
    del subImage
    _ = gc.collect()
    # 为了节省空间，删除全球图像
    os.remove(fn)

  3%|▎         | 1/34 [00:08<04:38,  8.45s/it]

outTiff: /home/mw/work/2. clipped dataset/F101992.v4b_web.stable_lights.avg_vis.tif


  6%|▌         | 2/34 [00:17<04:37,  8.68s/it]

outTiff: /home/mw/work/2. clipped dataset/F101993.v4b_web.stable_lights.avg_vis.tif


  9%|▉         | 3/34 [00:27<04:36,  8.93s/it]

...

outTiff: /home/mw/work/2. clipped dataset/F182011.v4c_web.stable_lights.avg_vis.tif


 97%|█████████▋| 33/34 [03:55<00:02,  2.58s/it]

outTiff: /home/mw/work/2. clipped dataset/F182012.v4c_web.stable_lights.avg_vis.tif


100%|██████████| 34/34 [03:56<00:00,  6.95s/it]

outTiff: /home/mw/work/2. clipped dataset/F182013.v4c_web.stable_lights.avg_vis.tif


### 矫正F14

In [7]:
# 定义函数：将多个numpy array合并成一个长array
@retry()
def concat(listArr:"a list of 2d array",flatten:bool=True):
    arrConcat = np.concatenate(listArr,axis=0)
    if flatten:
        return arrConcat.flatten()
    return arrConcat

In [8]:
# 图像路径
fnF121997 = glob.glob('/home/mw/work/2. clipped dataset/'+'*F121997*tif')[0]
fnF121998 = glob.glob('/home/mw/work/2. clipped dataset/'+'*F121998*tif')[0]
fnF121999 = glob.glob('/home/mw/work/2. clipped dataset/'+'*F121999*tif')[0]
fnF141997 = glob.glob('/home/mw/work/2. clipped dataset/'+'*F141997*tif')[0]
fnF141998 = glob.glob('/home/mw/work/2. clipped dataset/'+'*F141998*tif')[0]
fnF141999 = glob.glob('/home/mw/work/2. clipped dataset/'+'*F141999*tif')[0]
# 读取F12的图像
arrF121997 = readTiff(fnF121997)[0]
print('{0} has been read.'.format('arrF121997'))
arrF121998 = readTiff(fnF121998)[0]
print('{0} has been read.'.format('arrF121998'))
arrF121999 = readTiff(fnF121999)[0]
print('{0} has been read.'.format('arrF121999'))
# 把arrF121997、arrF121998、arrF121999合并成一维数组
arrF12 = concat([arrF121997,arrF121998,arrF121999])
print('arrF121997,arrF121998,arrF121999 has been concated')
del arrF121997,arrF121998,arrF121999
_ = gc.collect()
# 把arrF141997、arrF141998、arrF141999合并成一维数组
arrF141997 = readTiff(fnF141997)[0]
print('{0} has been read.'.format('arrF141997'))
arrF141998 = readTiff(fnF141998)[0]
print('{0} has been read.'.format('arrF141998'))
arrF141999 = readTiff(fnF141999)[0]
print('{0} has been read.'.format('arrF141999'))
# 把arrF141997、arrF141998、arrF141999合并成一维数组
arrF14 = concat([arrF141997,arrF141998,arrF141999])
print('arrF141997,arrF141998,arrF141999 has been concated')
del arrF141997,arrF141998,arrF141999
_ = gc.collect()

arrF121997 has been read.
arrF121998 has been read.
arrF121999 has been read.
arrF121997,arrF121998,arrF121999 has been concated
arrF141997 has been read.
arrF141998 has been read.
arrF141999 has been read.
arrF141997,arrF141998,arrF141999 has been concated


In [9]:
# 定义函数：计算模型的决定系数R2
@retry()
def R2(paras,x,yTrue):
    #
    model = np.poly1d(paras)
    yPred = model(x)
    R2 = 1 - np.sum((yTrue - yPred)**2) / np.sum((yTrue - np.mean(yTrue))**2)
    return R2

# 定义函数：通过二次多项式拟合参考图像refArr与待定标图像waitArr之间的关系
#@retry()
def fitParameter(waitArr,refArr,xlabel='F14',ylabel='F12'):
    # 定义亮像元的掩膜mask，它指示参考图像和待定标图像中都亮的区域
    mask0 = waitArr>0
    mask1 = refArr>0
    mask = mask0 * mask1
    del mask0,mask1
    _ = gc.collect()
    # 应用mask
    x = waitArr[mask==1.]
    y = refArr[mask==1.]
    del mask
    _ = gc.collect()
    # 用二次多项式拟合参考图像和待定标图像之间的关系
    paras = np.polyfit(x, y, 2)
    # 绘图显示
    ## 点
    plt.figure(figsize=(5,5))
    from matplotlib.colors import LogNorm
    _ = plt.hist2d(x,y, bins=200, norm=LogNorm())
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    ## 线 
    line  = []
    for i in range(64):
        line.append(np.poly1d(paras)(i))
    plt.plot(list(range(64)),line,'-')
    ## 参数
    plt.text(70, 10, 'w2={0}\n\nw1={1}\n\nw0={2}'.format(paras[0],paras[1],paras[2]))
    r_sq = R2(paras,x,y)
    plt.text(70, 4, 'R2={0}'.format(r_sq))
    del x,y
    _ = gc.collect()
    # 返回参数和R2
    return paras,r_sq

In [10]:
waitArr = arrF14
refArr = arrF12
paras,r_sq = fitParameter(waitArr,refArr)
print('parameters: {0}'.format(paras))
print('R2: {0}'.format(r_sq))
del waitArr,refArr,arrF14,arrF12
_ = gc.collect()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: RankWarning: Polyfit may be poorly conditioned
  This is separate from the ipykernel package so we can avoid doing imports until


parameters: [0.00964711 0.48680182 5.28645081]
R2: 0.9085130915045738


<Figure size 360x360 with 1 Axes>

In [11]:
# 定义函数：根据拟合的参数对待定标图像进行定标
def calibrate(waitArr,paras):
    model = np.poly1d(paras)
    waitArr[waitArr!=0.] = model(waitArr[waitArr!=0.])
    waitArr[waitArr<0.] = 0.
    waitArr[waitArr>63.] = 63.
    return waitArr

# 定义函数：输出图像
def outTif(arr,outFn,proj,trans):
    # 创建driver
    nrow = arr.shape[0]
    ncol = arr.shape[1]
    driver = gdal.GetDriverByName("GTiff")
    outdata = driver.Create(outFn,ncol,nrow,1, gdal.GDT_Float32)
    outband = outdata.GetRasterBand(1)
    # 写出数组
    outband.WriteArray(arr)
    # 添加地理参考
    im_proj = proj
    im_geotrans = trans
    outdata.SetGeoTransform(im_geotrans)
    outdata.SetProjection(im_proj)
    driver = None
    outdata = None
    outband = None


In [12]:
# 创建定标图像的输出路径
calibratedDir = '/home/mw/work/3. calibrated dataset/'
if not os.path.isdir(calibratedDir):
    os.mkdir(calibratedDir)

# 收集F14所有的图像
F14s = ['F141997','F141998','F141999','F142000','F142001','F142002','F142003']

# 遍历每一幅图像
for F14xxxx in F14s:
    # 读取图像，对其进行定标
    fnF14xxxx = glob.glob('/home/mw/work/2. clipped dataset/' + '*' + F14xxxx + '*tif')[0]
    arrF14xxxx,proj,trans = readTiff(fnF14xxxx)
    arrF14xxxx = calibrate(arrF14xxxx,paras)
    # 将定标的F14xxxx图像输出
    outFn = calibratedDir + os.path.basename(fnF14xxxx)
    outTif(arrF14xxxx,outFn,proj,trans)
    print('{0} has been output'.format(outFn))
    del arrF14xxxx
    _ = gc.collect()

/home/mw/work/3. calibrated dataset/F141997.v4b_web.stable_lights.avg_vis.tif has been output
/home/mw/work/3. calibrated dataset/F141998.v4b_web.stable_lights.avg_vis.tif has been output
/home/mw/work/3. calibrated dataset/F141999.v4b_web.stable_lights.avg_vis.tif has been output
/home/mw/work/3. calibrated dataset/F142000.v4b_web.stable_lights.avg_vis.tif has been output
/home/mw/work/3. calibrated dataset/F142001.v4b_web.stable_lights.avg_vis.tif has been output
/home/mw/work/3. calibrated dataset/F142002.v4b_web.stable_lights.avg_vis.tif has been output
/home/mw/work/3. calibrated dataset/F142003.v4b_web.stable_lights.avg_vis.tif has been output


In [13]:
# 找到所有F10卫星的图像
F10s = glob.glob('/home/mw/work/2. clipped dataset/' + 'F10*tif')
F10s.sort()
# 找到所有F12卫星的图像
F12s = glob.glob('/home/mw/work/2. clipped dataset/' + 'F12*tif')
F12s.sort()
# 依次拷贝到/3. calibrated dataset/下面
for f1nxxxx in tqdm(F10s + F12s):
    shutil.copy(f1nxxxx,  '/home/mw/work/3. calibrated dataset/'+os.path.basename(f1nxxxx))

100%|██████████| 9/9 [00:08<00:00,  1.08it/s]


In [14]:
# 定义函数：灯光亮度求和
def sumNTL(arr):
    sumNTL = np.sum(arr)
    return sumNTL

# 定义函数：创建dataframe以保存灯光总和值
def createDF():
    df = pd.DataFrame({"F10":[np.NAN for i in range(1992,2014)],
                       "F12":[np.NAN for i in range(1992,2014)],
                       "F14":[np.NAN for i in range(1992,2014)],
                       "F15":[np.NAN for i in range(1992,2014)],
                       "F16":[np.NAN for i in range(1992,2014)],
                       "F18":[np.NAN for i in range(1992,2014)],},index=range(1992,2014))
    return df

In [15]:
# 统计F10、F12、F14卫星的定标以前的数据（位于/2. clipped dataset/下）的灯光总和
dfSum_beforeCalibration = createDF()
# 遍历每一张图，进行求和计算
fnF10xxxx = glob.glob('/home/mw/work/2. clipped dataset/F10*tif')
fnF12xxxx = glob.glob('/home/mw/work/2. clipped dataset/F12*tif')
fnF14xxxx = glob.glob('/home/mw/work/2. clipped dataset/F14*tif')
for fn in fnF10xxxx + fnF12xxxx + fnF14xxxx:
    # 读取图像计算总和
    arr = readTiff(fn)[0]
    sumValue = sumNTL(arr)
    # 保存到dataframe的相应位置
    fnn = os.path.basename(fn)[0:3]
    xxxx = int(os.path.basename(fn)[3:7])
    dfSum_beforeCalibration.loc[xxxx,fnn] = sumValue
    del arr
    gc.collect()

In [16]:
# 统计F10、F12、F14卫星的定标以后的数据（位于/3. calibrated dataset/下）的灯光总和
dfSum_afterCalibration = createDF()
# 遍历每一张图，进行求和计算
fnF10xxxx = glob.glob('/home/mw/work/3. calibrated dataset/F10*tif')
fnF12xxxx = glob.glob('/home/mw/work/3. calibrated dataset/F12*tif')
fnF14xxxx = glob.glob('/home/mw/work/3. calibrated dataset/F14*tif')
for fn in fnF10xxxx + fnF12xxxx + fnF14xxxx:
    # 读取图像计算总和
    arr = readTiff(fn)[0]
    sumValue = sumNTL(arr)
    # 保存到dataframe的相应位置
    fnn = os.path.basename(fn)[0:3]
    xxxx = int(os.path.basename(fn)[3:7])
    dfSum_afterCalibration.loc[xxxx,fnn] = sumValue
    del arr
    gc.collect()

In [17]:
dfSum_beforeCalibration.plot(kind='line',figsize=[10,3],title='sum of light values !BEFORE! calibration')
dfSum_afterCalibration.plot(kind='line',figsize=[10,3],title='sum of light values !AFTER! calibration')

<AxesSubplot:title={'center':'sum of light values !AFTER! calibration'}>

<Figure size 720x216 with 1 Axes>

<Figure size 720x216 with 1 Axes>

### 矫正F15

In [18]:
# 图像路径
fnF142003 = glob.glob('/home/mw/work/3. calibrated dataset/'+'*F142003*tif')[0]
fnF152003 = glob.glob('/home/mw/work/2. clipped dataset/'+'*F152003*tif')[0]
# 读取F142003的图像
arrF142003 = readTiff(fnF142003)[0]
# 把F142003数组展成一维数组
arrF142003 = concat([arrF142003])
# 读取F152003的图像
arrF152003 = readTiff(fnF152003)[0]
# 把F152003数组展成一维数组
arrF152003 = concat([arrF152003])

In [19]:
waitArr = arrF152003
refArr = arrF142003
paras,r_sq = fitParameter(waitArr,refArr,xlabel='F152003',ylabel='F142003')
print('parameters: {0}'.format(paras))
print('R2: {0}'.format(r_sq))
del waitArr,refArr,arrF152003,arrF142003
_ = gc.collect()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: RankWarning: Polyfit may be poorly conditioned
  This is separate from the ipykernel package so we can avoid doing imports until


parameters: [0.00941815 0.49648862 6.13129568]
R2: 0.9481293857097626


<Figure size 360x360 with 1 Axes>

In [20]:
# 输出路径
calibratedDir = '/home/mw/work/3. calibrated dataset/'
if not os.path.isdir(calibratedDir):
    os.mkdir(calibratedDir)

# 收集F14所有的图像
F15s = ['F152003','F152004','F152005','F152006','F152007']

# 遍历每一幅图像
for F15xxxx in F15s:
    # 读取图像，对其进行定标
    fnF15xxxx = glob.glob('/home/mw/work/2. clipped dataset/' + '*' + F15xxxx + '*tif')[0]
    arrF15xxxx,proj,trans = readTiff(fnF15xxxx)
    arrF15xxxx = calibrate(arrF15xxxx,paras)
    # 将定标的F15xxxx图像输出
    outFn = calibratedDir + os.path.basename(fnF15xxxx)
    outTif(arrF15xxxx,outFn,proj,trans)
    print('{0} has been output'.format(outFn))
    del arrF15xxxx
    _ = gc.collect()

/home/mw/work/3. calibrated dataset/F152003.v4b_web.stable_lights.avg_vis.tif has been output
/home/mw/work/3. calibrated dataset/F152004.v4b_web.stable_lights.avg_vis.tif has been output
/home/mw/work/3. calibrated dataset/F152005.v4b_web.stable_lights.avg_vis.tif has been output
/home/mw/work/3. calibrated dataset/F152006.v4b_web.stable_lights.avg_vis.tif has been output
/home/mw/work/3. calibrated dataset/F152007.v4b_web.stable_lights.avg_vis.tif has been output


In [21]:
# 找到所有F15卫星的图像
F15s =  glob.glob('/home/mw/work/2. clipped dataset/' + 'F152000*tif')+ \
        glob.glob('/home/mw/work/2. clipped dataset/' + 'F152001*tif')+ \
        glob.glob('/home/mw/work/2. clipped dataset/' + 'F152002*tif')
F15s.sort()
# 依次拷贝到/3. calibrated dataset/下面
for f1nxxxx in tqdm(F15s):
    shutil.copy(f1nxxxx,  '/home/mw/work/3. calibrated dataset/'+os.path.basename(f1nxxxx))

100%|██████████| 3/3 [00:03<00:00,  1.03s/it]


In [22]:
# 统计F10、F12、F14、F15卫星的定标以前的数据（位于/2. clipped dataset/下）的灯光总和
dfSum_beforeCalibration = createDF()
# 遍历每一张图，进行求和计算
fnF10xxxx = glob.glob('/home/mw/work/2. clipped dataset/F10*tif')
fnF12xxxx = glob.glob('/home/mw/work/2. clipped dataset/F12*tif')
fnF14xxxx = glob.glob('/home/mw/work/2. clipped dataset/F14*tif')
fnF15xxxx = glob.glob('/home/mw/work/2. clipped dataset/F15*tif')
for fn in fnF10xxxx + fnF12xxxx + fnF14xxxx+ fnF15xxxx:
    # 读取图像计算总和
    arr = readTiff(fn)[0]
    sumValue = sumNTL(arr)
    # 保存到dataframe的相应位置
    fnn = os.path.basename(fn)[0:3]
    xxxx = int(os.path.basename(fn)[3:7])
    dfSum_beforeCalibration.loc[xxxx,fnn] = sumValue
    del arr
    gc.collect()

In [23]:
# 统计F10、F12、F14卫星的定标以后的数据（位于/3. calibrated dataset/下）的灯光总和
dfSum_afterCalibration = createDF()
# 遍历每一张图，进行求和计算
fnF10xxxx = glob.glob('/home/mw/work/3. calibrated dataset/F10*tif')
fnF12xxxx = glob.glob('/home/mw/work/3. calibrated dataset/F12*tif')
fnF14xxxx = glob.glob('/home/mw/work/3. calibrated dataset/F14*tif')
fnF15xxxx = glob.glob('/home/mw/work/3. calibrated dataset/F15*tif')
for fn in fnF10xxxx + fnF12xxxx + fnF14xxxx + fnF15xxxx:
    # 读取图像计算总和
    arr = readTiff(fn)[0]
    sumValue = sumNTL(arr)
    # 保存到dataframe的相应位置
    fnn = os.path.basename(fn)[0:3]
    xxxx = int(os.path.basename(fn)[3:7])
    dfSum_afterCalibration.loc[xxxx,fnn] = sumValue
    del arr
    gc.collect()

In [24]:
dfSum_beforeCalibration.plot(kind='line',figsize=[10,3],title='sum of light values !BEFORE! calibration')
dfSum_afterCalibration.plot(kind='line',figsize=[10,3],title='sum of light values !AFTER! calibration')

<AxesSubplot:title={'center':'sum of light values !AFTER! calibration'}>

<Figure size 720x216 with 1 Axes>

<Figure size 720x216 with 1 Axes>

### 矫正F16

In [25]:
# 图像路径
fnF152005 = glob.glob('/home/mw/work/3. calibrated dataset/'+'*F152005*tif')[0]
fnF162005 = glob.glob('/home/mw/work/2. clipped dataset/'+'*F162005*tif')[0]
# 读取F152005的图像
arrF152005 = readTiff(fnF152005)[0]
# 把F152005数组展成一维数组
arrF152005 = concat([arrF152005])
# 读取F162005的图像
arrF162005 = readTiff(fnF162005)[0]
# 把F162005数组展成一维数组
arrF162005 = concat([arrF162005])

In [26]:
waitArr = arrF162005
refArr = arrF152005
paras,r_sq = fitParameter(waitArr,refArr,xlabel='F162005',ylabel='F152005')
print('parameters: {0}'.format(paras))
print('R2: {0}'.format(r_sq))
del waitArr,refArr,arrF162005,arrF152005
_ = gc.collect()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: RankWarning: Polyfit may be poorly conditioned
  This is separate from the ipykernel package so we can avoid doing imports until


parameters: [0.00863814 0.47204778 6.30313554]
R2: 0.9603672511875629


<Figure size 360x360 with 1 Axes>

In [27]:
calibratedDir = '/home/mw/work/3. calibrated dataset/'
if not os.path.isdir(calibratedDir):
    os.mkdir(calibratedDir)

# 收集F14所有的图像
F16s = ['F162004','F162005','F162006','F162007','F162008','F162009']

# 遍历每一幅图像
for F16xxxx in F16s:
    # 读取图像，对其进行定标
    fnF16xxxx = glob.glob('/home/mw/work/2. clipped dataset/' + '*' + F16xxxx + '*tif')[0]
    arrF16xxxx,proj,trans = readTiff(fnF16xxxx)
    arrF16xxxx = calibrate(arrF16xxxx,paras)
    # 将定标的F16xxxx图像输出
    outFn = calibratedDir + os.path.basename(fnF16xxxx)
    outTif(arrF16xxxx,outFn,proj,trans)
    print('{0} has been output'.format(outFn))
    del arrF16xxxx
    _ = gc.collect()

/home/mw/work/3. calibrated dataset/F162004.v4b_web.stable_lights.avg_vis.tif has been output
/home/mw/work/3. calibrated dataset/F162005.v4b_web.stable_lights.avg_vis.tif has been output
/home/mw/work/3. calibrated dataset/F162006.v4b_web.stable_lights.avg_vis.tif has been output
/home/mw/work/3. calibrated dataset/F162007.v4b_web.stable_lights.avg_vis.tif has been output
/home/mw/work/3. calibrated dataset/F162008.v4b_web.stable_lights.avg_vis.tif has been output
/home/mw/work/3. calibrated dataset/F162009.v4b_web.stable_lights.avg_vis.tif has been output


In [28]:
# 统计F10、F12、F14、F15、F16卫星的定标以前的数据（位于/2. clipped dataset/下）的灯光总和
dfSum_beforeCalibration = createDF()
# 遍历每一张图，进行求和计算
fnF10xxxx = glob.glob('/home/mw/work/2. clipped dataset/F10*tif')
fnF12xxxx = glob.glob('/home/mw/work/2. clipped dataset/F12*tif')
fnF14xxxx = glob.glob('/home/mw/work/2. clipped dataset/F14*tif')
fnF15xxxx = glob.glob('/home/mw/work/2. clipped dataset/F15*tif')
fnF16xxxx = glob.glob('/home/mw/work/2. clipped dataset/F16*tif')
for fn in fnF10xxxx + fnF12xxxx + fnF14xxxx+ fnF15xxxx+ fnF16xxxx:
    # 读取图像计算总和
    arr = readTiff(fn)[0]
    sumValue = sumNTL(arr)
    # 保存到dataframe的相应位置
    fnn = os.path.basename(fn)[0:3]
    xxxx = int(os.path.basename(fn)[3:7])
    dfSum_beforeCalibration.loc[xxxx,fnn] = sumValue
    del arr
    gc.collect()

In [29]:
# 统计F10、F12、F14、F15、F16卫星的定标以后的数据（位于/3. calibrated dataset/下）的灯光总和
dfSum_afterCalibration = createDF()
# 遍历每一张图，进行求和计算
fnF10xxxx = glob.glob('/home/mw/work/3. calibrated dataset/F10*tif')
fnF12xxxx = glob.glob('/home/mw/work/3. calibrated dataset/F12*tif')
fnF14xxxx = glob.glob('/home/mw/work/3. calibrated dataset/F14*tif')
fnF15xxxx = glob.glob('/home/mw/work/3. calibrated dataset/F15*tif')
fnF16xxxx = glob.glob('/home/mw/work/3. calibrated dataset/F16*tif')
for fn in fnF10xxxx + fnF12xxxx + fnF14xxxx + fnF15xxxx+ fnF16xxxx:
    # 读取图像计算总和
    arr = readTiff(fn)[0]
    sumValue = sumNTL(arr)
    # 保存到dataframe的相应位置
    fnn = os.path.basename(fn)[0:3]
    xxxx = int(os.path.basename(fn)[3:7])
    dfSum_afterCalibration.loc[xxxx,fnn] = sumValue
    del arr
    gc.collect()

In [30]:
dfSum_beforeCalibration.plot(kind='line',figsize=[10,3],title='sum of light values !BEFORE! calibration')
dfSum_afterCalibration.plot(kind='line',figsize=[10,3],title='sum of light values !AFTER! calibration')

<AxesSubplot:title={'center':'sum of light values !AFTER! calibration'}>

<Figure size 720x216 with 1 Axes>

<Figure size 720x216 with 1 Axes>

#### 校正F182010

In [31]:
# 图像路径
fnF162009 = glob.glob('/home/mw/work/3. calibrated dataset/'+'*F162009*tif')[0]
fnF182010 = glob.glob('/home/mw/work/2. clipped dataset/'+'*F182010*tif')[0]
# 读取F162009的图像
arrF162009 = readTiff(fnF162009)[0]
# 把F162009数组展成一维数组
arrF162009 = concat([arrF162009])
# 读取F182010的图像
arrF182010 = readTiff(fnF182010)[0]
# 把F182010数组展成一维数组
arrF182010 = concat([arrF182010])

In [33]:
waitArr = arrF182010
refArr = arrF162009
paras,r_sq = fitParameter(waitArr,refArr,xlabel='F182010',ylabel='F162009')
print('parameters: {0}'.format(paras))
print('R2: {0}'.format(r_sq))
del waitArr,refArr,arrF182010,arrF162009
_ = gc.collect()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: RankWarning: Polyfit may be poorly conditioned
  This is separate from the ipykernel package so we can avoid doing imports until


parameters: [0.00711127 0.34203734 5.3320811 ]
R2: 0.9080354273319244


<Figure size 360x360 with 1 Axes>

In [34]:
# 输出路径
calibratedDir = '/home/mw/work/3. calibrated dataset/'
if not os.path.isdir(calibratedDir):
    os.mkdir(calibratedDir)

# 读取图像，对其进行定标
fnF182010 = glob.glob('/home/mw/work/2. clipped dataset/' + '*F182010*tif')[0]
arrF182010,proj,trans = readTiff(fnF182010)
arrF182010 = calibrate(arrF182010,paras)
# 将定标的F182010图像输出
outFn = calibratedDir + os.path.basename(fnF182010)
outTif(arrF182010,outFn,proj,trans)
print('{0} has been output'.format(outFn))
del arrF182010
_ = gc.collect()

/home/mw/work/3. calibrated dataset/F182010.v4d_web.stable_lights.avg_vis.tif has been output


In [35]:
# 找到所有F18卫星的图像
F18s =  glob.glob('/home/mw/work/2. clipped dataset/' + 'F182011*tif')+ \
        glob.glob('/home/mw/work/2. clipped dataset/' + 'F182012*tif')+ \
        glob.glob('/home/mw/work/2. clipped dataset/' + 'F182013*tif')
F18s.sort()
# 依次拷贝到/3. calibrated dataset/下面
for f1nxxxx in tqdm(F18s):
    shutil.copy(f1nxxxx,  '/home/mw/work/3. calibrated dataset/'+os.path.basename(f1nxxxx))

100%|██████████| 3/3 [00:02<00:00,  1.08it/s]


In [36]:
# 统计F10、F12、F14、F15、F18卫星的定标以前的数据（位于/2. clipped dataset/下）的灯光总和
dfSum_beforeCalibration = createDF()
# 遍历每一张图，进行求和计算
fnF10xxxx = glob.glob('/home/mw/work/2. clipped dataset/F10*tif')
fnF12xxxx = glob.glob('/home/mw/work/2. clipped dataset/F12*tif')
fnF14xxxx = glob.glob('/home/mw/work/2. clipped dataset/F14*tif')
fnF15xxxx = glob.glob('/home/mw/work/2. clipped dataset/F15*tif')
fnF16xxxx = glob.glob('/home/mw/work/2. clipped dataset/F16*tif')
fnF18xxxx = glob.glob('/home/mw/work/2. clipped dataset/F18*tif')
for fn in fnF10xxxx + fnF12xxxx + fnF14xxxx+ fnF15xxxx+ fnF16xxxx+ fnF18xxxx:
    # 读取图像计算总和
    arr = readTiff(fn)[0]
    sumValue = sumNTL(arr)
    # 保存到dataframe的相应位置
    fnn = os.path.basename(fn)[0:3]
    xxxx = int(os.path.basename(fn)[3:7])
    dfSum_beforeCalibration.loc[xxxx,fnn] = sumValue
    del arr
    gc.collect()

In [37]:
# 统计F10、F12、F14、F15、F16、F18卫星的定标以后的数据（位于/3. calibrated dataset/下）的灯光总和
dfSum_afterCalibration = createDF()
# 遍历每一张图，进行求和计算
fnF10xxxx = glob.glob('/home/mw/work/3. calibrated dataset/F10*tif')
fnF12xxxx = glob.glob('/home/mw/work/3. calibrated dataset/F12*tif')
fnF14xxxx = glob.glob('/home/mw/work/3. calibrated dataset/F14*tif')
fnF15xxxx = glob.glob('/home/mw/work/3. calibrated dataset/F15*tif')
fnF16xxxx = glob.glob('/home/mw/work/3. calibrated dataset/F16*tif')
fnF18xxxx = glob.glob('/home/mw/work/3. calibrated dataset/F18*tif')
for fn in fnF10xxxx + fnF12xxxx + fnF14xxxx + fnF15xxxx+ fnF16xxxx+ fnF18xxxx:
    # 读取图像计算总和
    arr = readTiff(fn)[0]
    sumValue = sumNTL(arr)
    # 保存到dataframe的相应位置
    fnn = os.path.basename(fn)[0:3]
    xxxx = int(os.path.basename(fn)[3:7])
    dfSum_afterCalibration.loc[xxxx,fnn] = sumValue
    del arr
    gc.collect()

In [38]:
dfSum_beforeCalibration.plot(kind='line',figsize=[10,3],title='sum of light values !BEFORE! calibration')
dfSum_afterCalibration.plot(kind='line',figsize=[10,3],title='sum of light values !AFTER! calibration')

<AxesSubplot:title={'center':'sum of light values !AFTER! calibration'}>

<Figure size 720x216 with 1 Axes>

<Figure size 720x216 with 1 Axes>

##  制作城市发展轨迹图-北京

In [39]:
_ = gc.collect()

In [40]:
# 选择以下数据制作动态图
fnnxxxxs = ['F101992','F101993','F101994',
            'F121995','F121996','F121997','F121998',
            'F141999','F142000','F142001','F142002',
            'F152003','F152004','F152005','F152006',
            'F162007','F162008','F162009',
            'F182010','F182011','F182012','F182013']

In [41]:
# 定义函数：创建每年的png图片
def createPNG(arr,outFn):
    fig = plt.figure(figsize=(20,20))
    plt.imshow(arr,vmin=0, vmax=63)
    plt.savefig(outFn)
    plt.close(fig)
    del fig
    _ = gc.collect()

In [42]:
# 创建路径
outDir = "/home/mw/work/4. gif/"
if not os.path.isdir(outDir):
    os.mkdir(outDir)

# 定义城市及其周边区域的坐标(完成基础作业只需要改变下面的注释内容即可)
cities_coords = {'Beijing':[(1760,1412),(1840,1576)]}

# 遍历每个城市及其周边区域，输出每年的图像
for city,coord in cities_coords.items():
    # 遍历定标的每一年的图像
    for fnnxxxx in tqdm(fnnxxxxs):
        # 读取数据
        fullname_fnnxxxx = glob.glob('/home/mw/work/3. calibrated dataset/'+'*'+fnnxxxx+'*tif')[0]
        arr,proj,trans = readTiff(fullname_fnnxxxx)
        # 裁剪图像
        subArr,subProj,subTrans = clipImg(arr,proj,trans,coord[0],coord[1])
        del arr
        _ = gc.collect()
        # 输出图像
        sub_outDir = outDir + city + '/'
        if not os.path.isdir(sub_outDir):
            os.mkdir(sub_outDir)
        createPNG(subArr,sub_outDir + os.path.basename(fnnxxxx))
        del subArr
        _ = gc.collect()

100%|██████████| 22/22 [00:09<00:00,  2.20it/s]


In [43]:
# 定义函数：根据每年的png图片，创建gif图片
def create_gif(dirCity, gif_name):
    image_list = glob.glob(dirCity+'*png')
    image_list.sort()
    #
    frames = []
    for image_name in image_list:
        frames.append(imageio.imread(image_name))
    imageio.mimsave(gif_name, frames, 'GIF', duration = 0.2)
    return

In [44]:
# 为每个城市创建gif图片
for city in ['Beijing']:
    dir_city = '/home/mw/work/4. gif/' + city + '/'
    gif_name = '/home/mw/work/4. gif/' + city + '.gif'
    create_gif(dir_city, gif_name)

In [45]:
from IPython.display import Image
Image(open('/home/mw/work/4. gif/Beijing.gif','rb').read())

##  制作城市发展轨迹图-广州

In [46]:
# 创建路径
outDir = "/home/mw/work/4. gif/"
if not os.path.isdir(outDir):
    os.mkdir(outDir)

# 定义城市及其周边区域的坐标(完成基础作业只需要改变下面的注释内容即可)
cities_coords = {'Guangzhou':[(3723,1033),(3943,1269)]}

# 遍历每个城市及其周边区域，输出每年的图像
for city,coord in cities_coords.items():
    # 遍历定标的每一年的图像
    for fnnxxxx in tqdm(fnnxxxxs):
        # 读取数据
        fullname_fnnxxxx = glob.glob('/home/mw/work/3. calibrated dataset/'+'*'+fnnxxxx+'*tif')[0]
        arr,proj,trans = readTiff(fullname_fnnxxxx)
        # 裁剪图像
        subArr,subProj,subTrans = clipImg(arr,proj,trans,coord[0],coord[1])
        del arr
        _ = gc.collect()
        # 输出图像
        sub_outDir = outDir + city + '/'
        if not os.path.isdir(sub_outDir):
            os.mkdir(sub_outDir)
        createPNG(subArr,sub_outDir + os.path.basename(fnnxxxx))
        del subArr
        _ = gc.collect()

100%|██████████| 22/22 [00:11<00:00,  1.98it/s]


In [47]:
# 为每个城市创建gif图片
for city in ['Guangzhou']:
    dir_city = '/home/mw/work/4. gif/' + city + '/'
    gif_name = '/home/mw/work/4. gif/' + city + '.gif'
    create_gif(dir_city, gif_name)

In [48]:
from IPython.display import Image
Image(open('/home/mw/work/4. gif/Guangzhou.gif','rb').read())

## 总结

通过一个小的Demo加深了对于《如何给灯光数据定标》这一块技能的知识点的学习，通过每一块的代码以及注释对于过程和一个整体的流程有了一个了解。后续有时间打算继续试着训练其他的数据集尝试看看能否利用所学所了解的代码做出新的知识

整个学习过程中最初有一个困惑就是对于区域坐标是如何获取，如果不是中国东部这块区域，或者不是北京上海广州这三个城市，需要如何去获取对应的坐标，这个也是一个在课后需要去了解学习的一个点。后来学习群里询问得知GDAL库可以实现转换，于是自己网上找了一些代码，但是会有一些报错，自己没法解决，后来课件也更新了转换代码，通过学习课件的转换代码后学习了解了整个过程

## 附录  使用GDAL库进行坐标转换

In [1]:
from osgeo import gdal
from osgeo import osr
import numpy as np


def geo2imagexy(dataset, x, y):
    '''
    根据GDAL的六参数模型将给定的投影或地理坐标转为影像图像坐标（行列号）
    :param dataset: GDAL地理数据
    :param x: 投影或地理坐标x
    :param y: 投影或地理坐标y
    :return: r行号,c列号
    '''
    trans = dataset.GetGeoTransform()
    c = (x - trans[0]) / trans[1]
    r = (y - trans[3]) / trans[5]
    return int(r), int(c)


if __name__ == '__main__':
    gdal.AllRegister()
    dataset = gdal.Open(
        r"/home/mw/work/2. clipped dataset/F101992.v4b_web.stable_lights.avg_vis.tif")
    print('数据投影：')
    print(dataset.GetProjection())
    print('数据的大小（行，列）：')
    print('(%s %s)' % (dataset.RasterYSize, dataset.RasterXSize))

    x1 = 120.51  # 上海区域最西边经度
    y1 = 31.53  # 上海区域最北边纬度

    x2 = 122.12  # 上海区域最东边经度
    y2 = 30.40  # 上海区域最南边纬度

    print('左上角地理坐标 -> 图像坐标：')
    coords_1 = geo2imagexy(dataset, x1, y1)
    print('(%s, %s)->(%s, %s)' % (x1, y1, coords_1[0], coords_1[1]))

    print('右下角地理坐标 -> 图像坐标：')
    coords_2 = geo2imagexy(dataset, x2, y2)
    print('(%s, %s)->(%s, %s)' % (x2, y2, coords_2[0], coords_2[1]))

数据投影：

数据的大小（行，列）：
(4538 4012)
左上角地理坐标 -> 图像坐标：
(120.51, 31.53)->(2783, 1991)
右下角地理坐标 -> 图像坐标：
(122.12, 30.4)->(2919, 2184)
